**Progetto Machine Learning**
---
**Giovanni D'Agostino, Marco Giorgi**

Fase Preliminare: 
- individuare la GPU Tesla-P100
- montare il drive contenente il codice e i dataset
- selezionare i path opportuni
- configurazione size window e batch 

In [ ]:
########## Tipo GPU allocata #############
!nvidia-smi

Mon Nov 30 18:14:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
import sys
drive.mount('Project_ML')
sys.path.append('/content/Project_ML/My Drive/ML/ProveFinali/Fridge')
path_main_test = '/content/Project_ML/My Drive/ML/ProveFinali/main_train.csv' #Insert main test
path_fridge_test = '/content/Project_ML/My Drive/ML/ProveFinali/fridge_train.csv' # Insert fridge test
window_size= 3999 #3999 NB!deve avere dimensione dispari
batch_size= 512  #512


Mounted at Project_ML


# **Load Dataset**



In [ ]:
import pandas as pd
import preprocessing 
import metrics
import generator
import stats_data
import tensorflow as tf
import numpy as np
df_main_test = pd.read_csv(path_main_test)
df_fridge_test = pd.read_csv(path_fridge_test)
model_fridge = tf.keras.models.load_model('/content/Project_ML/My Drive/ML/ProveFinali/Fridge/modelFinaleFridge', custom_objects=None) 

# **Preprocessing**

In [ ]:
main_power = preprocessing.load_data(df_main_test)
fridge_power= preprocessing.load_data(df_fridge_test)
main_power_padded = preprocessing.padding(main_power, window_size) 
fridge_power_padded = preprocessing.padding(fridge_power, window_size)
mmM= np.mean(main_power)
stdM= np.std(main_power)
mmF= np.mean(fridge_power)
stdF= np.std(fridge_power)
main_power_strd  = preprocessing.standardizzazione(main_power_padded, mmM, stdM)
fridge_power_strd = preprocessing.standardizzazione(fridge_power_padded, mmF, stdF)

In [ ]:
ground_truth = fridge_power + np.zeros(fridge_power.shape)

# **Test**

In [ ]:
generator = generator.DataGenerator(main_power_strd, fridge_power_strd, window_len=window_size, batch_len=batch_size, shuffle=False)
pred_fridge = model_fridge.predict(x=generator, steps=generator.__len__(), verbose=1)

In [ ]:
pred_fridge_reverse= preprocessing.reverse_strd(mmF, stdF, pred_fridge)

In [ ]:
f1_p= metrics.f1_score(ground_truth, pred_fridge_reverse)
print("F1:",f1_p)